# O script tem a função de obter informações gerais da empresa selecionada da **B3** e suas cotações em datas especifícas.
## Além de armazenar os dados em uma planilha do Excel.
### Importação das bibliotecas. 
### Interface gráfica feita pelo Tkinter, além das bibliotecas para manipular dados e criar gráficos


In [3]:
import tkinter as tk
from tkinter import messagebox
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime

In [4]:

# Função para buscar e processar dados da ação
def buscar_dados():
    global cotacaodf, ativo, datadre
    ativo = entrada_ativo.get()
    ativo = ativo.upper() + '.SA'
    
    try:
        # Uso do yfinance para buscar dados da ação
        papel = yf.Ticker(ativo)
        sobre = papel.info
        nome = sobre.get('longName', 'Nome não encontrado')
        regiao = sobre.get('timeZoneFullName', 'Time ZoneFullName not found')
        ultimodiv = sobre.get('lastDividendValue', 'último dividendo não encontrado')
        recomendacao = sobre.get('recommendationKey', 'recomendação de compra não encontrada')
        sites =  sobre.get('website', 'Site não encontrado')
        
        # Buscar dados de cotação
        cotacao = papel.history(period='1y')  # Exemplo: últimos 1 ano de dados
        maxcota = cotacao['Close'].max()
        mincota = cotacao['Close'].min()
        date_max = cotacao.index[cotacao['Close'] == maxcota][0]
        date_min = cotacao.index[cotacao['Close'] == mincota][0]
        date_max_str = date_max.strftime('%Y-%m-%d')
        date_min_str = date_min.strftime('%Y-%m-%d')
        
        # Criar DataFrame para cotação
        cotacaodf = pd.DataFrame(cotacao)
        cotacaodf['Date'] = cotacaodf.index
        cotacaodf = cotacaodf.rename(columns={'Open': 'Abertura', 'High': 'Máxima', 'Low': 'Mínima', 'Close': 'Fechamento'})
        cotacaodf['Date'] = cotacaodf['Date'].dt.tz_localize(None)
        
        # Mostrar informações na interface
        txt_resultado.config(state=tk.NORMAL)
        txt_resultado.delete('1.0', tk.END)
        txt_resultado.insert(tk.END, f'\n Informações de {ativo} \n')
        txt_resultado.insert(tk.END, f'Nome: {nome}\n')
        txt_resultado.insert(tk.END, f'Região: {regiao}\n')
        txt_resultado.insert(tk.END, f'Último dividendo: {ultimodiv}\n')
        txt_resultado.insert(tk.END, f'Recomendação:',recomendacao)
        txt_resultado.insert(tk.END, f'{ativo} apresentou preço máximo de {maxcota.round(2)} no dia {date_max_str}\n')
        txt_resultado.insert(tk.END, f'{ativo} apresentou preço mínimo de {mincota.round(2)} no dia {date_min_str}\n')
        txt_resultado.insert(tk.END, "Mais informações salvas na planilha....\n\n")
        txt_resultado.config(state=tk.DISABLED)
        
        # Criar planilha Excel com os dados
        planilha_excel = Workbook()
        planilhas = {
            'Informações': pd.DataFrame([nome, regiao, ultimodiv, recomendacao, sites]),
            'Cotação': cotacaodf
        }
        
        for nome_planilha, df in planilhas.items():
            ws = planilha_excel.create_sheet(title=nome_planilha)
            for linha in dataframe_to_rows(df, index=False, header=True):
                ws.append(linha)
        
        planilha_excel.remove(planilha_excel['Sheet'])
        nome_arquivo = 'dados_empresa.xlsx'
        planilha_excel.save(nome_arquivo)
        
    except Exception as e:
        messagebox.showerror('Erro', f'Ocorreu um erro ao buscar dados: {str(e)}')

In [5]:
def obterdre():
    print('Estou produzindo ainda...')


In [6]:
def criagrafico():
    global cotacaodf, ativo
    
    plt.figure(figsize=(4, 2))  # Define o tamanho da figura
    plt.plot(cotacaodf['Date'], cotacaodf['Fechamento'], linestyle='-', color='b', label='Fechamento')
    plt.title(f'Cotação de Fechamento de {ativo}')
    plt.xlabel('Data')
    plt.ylabel('Preço de Fechamento')
    plt.xticks(rotation=45)  # Rotaciona os rótulos do eixo x para melhor legibilidade
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Criar janela principal
root = tk.Tk()
root.title('Análise de Ações')

# Criar e posicionar elementos na janela
lbl_ativo = tk.Label(root, text='Digite a sigla da ação:')
lbl_ativo.pack(pady=10)

entrada_ativo = tk.Entry(root, width=30)
entrada_ativo.pack()

btn_buscar = tk.Button(root, text='Buscar Dados', command=buscar_dados)
btn_buscar.pack(pady=10)

btn_grafico = tk.Button(root, text='Gerar Gráfico', command=criagrafico)
btn_grafico.pack(pady=10)

btn_dre = tk.Button(root, text='Obter DRE',command=obterdre)
btn_dre.pack(pady=10)

txt_resultado = tk.Text(root, width=50, height=20, wrap=tk.WORD)
txt_resultado.pack(padx=10, pady=10)
txt_resultado.config(state=tk.DISABLED)

# Iniciar o loop principal da interface gráfica
root.mainloop()
